In [ ]:
import time, DM_Profiles, numpy, scipy
import pynbody as pyn
from pynbody import units
from matplotlib import pylab as plt
s = pyn.load('/media/tengiz/81498267-fb7c-4587-88ea-5c63aaf2cb66/Pynbody/testdata/g15784.lr.01024.gz')
s.physical_units()
h = s.halos()

In [ ]:
def model_prep(halo):
    """
    This prepares profile for model to take in
    """
    # centering to generate profile and placing particles back 
    with pyn.analysis.angmom.faceon(halo, cen_size  =  '10 kpc'):
        r_200 = float(pyn.analysis.halo.virial_radius(halo.d, overden = 200))
        
        # finding furthest particle position
        position = []
        for i in halo['pos']:
            position.append(numpy.sqrt(i[0]**2+i[1]**2+i[2]**2))
        furthest = max(position)
        
        max_radius = min(furhtest+1, r_200)
        # finding star half-mass radius
        stellar_profile = pyn.analysis.profile.Profile(halo.s, min = 0.01, max = max_length, ndim = 2, type = 'equaln', nbins = 10000)
        shm_radius = stellar_profile['rbins'][len(stellar_profile['rbins'])//2]

        profile = pyn.analysis.profile.Profile(halo.d, min = 2*max(halo.d['eps']), max = furthest+1, ndim = 3, type = 'log', nbins = 1000)
    
    # calculating steallar and halo mass
    
    sm = halo.s['mass'].sum()
    hm = halo['mass'].sum()
        
    return profile, sm, hm, shm_radius, r_200

In [ ]:
def den_plot(halo_profile, to_save = False):
    '''
    takes model profile object from DM_Profiles and plots den profile with a curve_fit
    '''
    
    fig, ax = plt.subplots() 
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_den), 'g--')
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_rho(numpy.array(halo_profile.radii), *halo_profile.params)), 'r-')
    ax.grid()
    ax.legend(('data','fit'))
    ax.set_title(halo_profile.name + ' ' + halo_profile.pmodel + ' density profile')
    ax.set_xlabel('$R$ [kpc]')
    ax.set_ylabel(r'$\rho$ [M$_{\odot}$ /kpc$^{3}$]')
    plt.errorbar(halo_profile.radii, numpy.power(10, halo_profile.log_den) , yerr = numpy.array(halo_profile.den_error), fmt = 'none')
    ax.set_yscale('log')
    ax.set_xscale('log')
    if to_save:
        plt.savefig('../Graphs/density_plot' + haloprofile.name +  '_density.jpg')
    plt.show()

In [ ]:
i = 1
halo = h[i]
start_time = time.time()
variables = model_prep(halo) 

In [ ]:
hp = DM_Profiles.model(*variables, 'halo_' +  str(i), pmodel = 'coreNFW')
print(hp.output())
den_plot(hp)
print(time.time() - start_time)

In [ ]:
'''
Question: do priors affect my fits for DC14, because I have a separate calculation for SMH and what do I do with 
The SHM relation (Moster, Naab & White 2013) presents a lognormal distribution around the mean relation
'''

In [ ]:
h[1].properties['time'].in_units('Gyr')